In [ ]:
import requests
from matchms.importing import load_from_msp
from matchms import set_matchms_logger_level
import pandas as pd
from rdkit import Chem

set_matchms_logger_level("ERROR")

api = 'https://spectus-demo.dyn.cloud.e-infra.cz/predictions'

In [ ]:
msp_file_path = "SWGDRUG_100.msp"
msp = load_from_msp(msp_file_path)
mz,intensity,smiles = zip(*[ (
    list(map(float,s.peaks.mz)),
    list(map(float,s.peaks.intensities)),
    s.metadata.get('canonical_smiles'))
    for s in msp ])
len(mz)

In [ ]:
topk = 20
idx = 7

response_post = requests.post(api, json={
    "mz": mz[idx],
    "intensity": intensity[idx],
    "topk" : topk,
    "smiles": "unknown"
})

if response_post.status_code == 202:
    print('POST request successful.')
    print('Response (POST):', response_post.json())
else:
    print('POST request failed with status code:', response_post.status_code)

created_post_id = response_post.json().get('id') if response_post.status_code == 202 else None

In [ ]:
result = None
if created_post_id:
    # Send a GET request
    resp = requests.get(f"{api}/{created_post_id}")

    if resp.status_code == 200:
        stat = resp.json()['status']
        print(stat)
        if stat == 'completed':
            result = resp.json()['result']
    else:
        print('HTTP:', resp.status_code)
else:
    print('No valid post ID was obtained from POST response for GET request.')

In [ ]:
r = pd.DataFrame(result.items(),columns=['SMILES','confidence'])\
    .sort_values(by='confidence',ascending=False)\
    .reset_index(drop=True)
r

In [ ]:
ground_truth = Chem.MolFromSmiles(smiles[idx])
ground_truth

In [ ]:
n = 1
Chem.MolFromSmiles(r.loc[n].SMILES)